In [ ]:
import os
import librosa
import numpy as np
import noisereduce as nr
import soundfile as sf
from pydub import AudioSegment
from pydub.silence import split_on_silence


def load_audio(file_path, target_sr=16000):
    if file_path.lower().endswith(".mp3"):
        audio = AudioSegment.from_file(file_path, format="mp3").set_channels(1)
        samples = np.array(audio.get_array_of_samples()).astype(np.float32) / 32768.0
        sr = audio.frame_rate
        y = librosa.resample(samples, orig_sr=sr, target_sr=target_sr)
        return y, target_sr
    else:
        y, sr = librosa.load(file_path, sr=target_sr)
        return y, sr


def normalize_volume(samples):
    return librosa.util.normalize(samples)


def reduce_noise(samples, sr):
    return nr.reduce_noise(y=samples, sr=sr)


# removing background noise
def bandpass_filter(samples, sr, low=80, high=8000):
    fft = librosa.stft(samples)
    freqs = librosa.fft_frequencies(sr=sr)
    mask = (freqs >= low) & (freqs <= high)
    fft[~mask, :] = 0
    return librosa.istft(fft)


def remove_silence_from_array(y, sr, silence_thresh=-35, min_silence_len=300):

    temp_path = "temp_for_silence.wav"
    sf.write(temp_path, y, sr)

    sound = AudioSegment.from_wav(temp_path)
    chunks = split_on_silence(
        sound,
        min_silence_len=min_silence_len,
        silence_thresh=silence_thresh,
        keep_silence=100,
    )

    if not chunks:
        print("No speech detected ")
        return y

    combined = AudioSegment.empty()
    for chunk in chunks:
        combined += chunk

    os.remove(temp_path)
    samples = np.array(combined.get_array_of_samples()).astype(np.float32) / 32768.0
    return samples


def preprocess_audio(file_path, output_path):
    print(f" Processing: {file_path}")
    try:
        y, sr = load_audio(file_path)

        y = reduce_noise(y, sr)
        y = normalize_volume(y)
        y = bandpass_filter(y, sr)
        y = remove_silence_from_array(y, sr)

        sf.write(output_path, y, sr)
        print(f"Saved to: {output_path}")
    except Exception as e:
        print(f" Error processing {file_path}: {e}")


def preprocess_folder(folder_path="data", output_folder="processed"):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    files = os.listdir(folder_path)
    print(f" Found files: {files}")

    for filename in files:
        if filename.lower().endswith((".wav", ".mp3")):
            input_path = os.path.join(folder_path, filename)
            output_path = os.path.join(output_folder, filename.replace(".mp3", ".wav"))
            preprocess_audio(input_path, output_path)


if __name__ == "__main__":
    preprocess_folder("data", "processed")

 Found files: ['common_voice_en_250.mp3']
 Processing: data/common_voice_en_250.mp3
Saved to: processed/common_voice_en_250.wav
